**This method involves analyzing the behavior of the users. It recommends movies based on the likes from users with similar taste in movies.**

In [1]:
import pandas as pd

## Data Preparation

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [33]:
movies = pd.read_csv('../datasets/ml-20m/movies.csv')
ratings = pd.read_csv('../datasets/ml-20m/ratings.csv')
df = movies.merge(ratings, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,9.449194e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,8.582755e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.339819e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,9.434979e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,1.230859e+09
...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,1.427744e+09
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,1.427745e+09
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,1.427745e+09
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,1.427745e+09


## Creating User-Movie DataFrame as in Item-Based

In [4]:
rating_counts = df.groupby('title').size().reset_index(name='count').sort_values(by='count', ascending=False)
rating_counts

,title,count
19154,Pulp Fiction (1994),67310
8905,Forrest Gump (1994),66172
21140,"Shawshank Redemption, The (1994)",63366
21360,"Silence of the Lambs, The (1991)",63299
12881,Jurassic Park (1993),59715
...,...,...
23790,The Italian Connection (1972),1
23788,The Iron Commissioner (1978),1
23787,The Invitation (2003),1
23786,The Invisible Frame (2009),1


Movies with less than 1000 ratings

In [42]:
rare_movies = rating_counts[rating_counts['count'] <= 1000]['title']
type(rare_movies)

pandas.core.series.Series

Movies with more than 1000 ratings

In [43]:
common_movies_df = df[~df['title'].isin(rare_movies)]
common_movies_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,9.449194e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,8.582755e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.339819e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,9.434979e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,1.230859e+09
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,1.411419e+09
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,1.411488e+09
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,1.412778e+09
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,1.411251e+09


Pivoting the DataFrame

In [44]:
users_movies_df = common_movies_df.pivot_table(index=['userId'], columns=['title'], values='rating')
users_movies_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

Selecting a User Randomly

In [101]:
user_random_id = int(pd.Series(users_movies_df.index).sample(1, random_state=45).values)

Creating a DataFrame of the selected user

In [46]:
user_random_df = users_movies_df[users_movies_df.index == user_random_id]
user_random_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

Finding the movies that the selected user has watched

In [10]:
user_random_watched_list = user_random_df.columns[user_random_df.notna().any()].tolist()
user_random_watched_list

['Ace Ventura: Pet Detective (1994)',
 'Ace Ventura: When Nature Calls (1995)',
 'Aladdin (1992)',
 'American President, The (1995)',
 'Apollo 13 (1995)',
 'Babe (1995)',
 'Bullets Over Broadway (1994)',
 'Clueless (1995)',
 'Disclosure (1994)',
 'Forrest Gump (1994)',
 'Four Weddings and a Funeral (1994)',
 'Home Alone (1990)',
 'Jurassic Park (1993)',
 'Like Water for Chocolate (Como agua para chocolate) (1992)',
 'Little Women (1994)',
 "Mr. Holland's Opus (1995)",
 'Mrs. Doubtfire (1993)',
 'Much Ado About Nothing (1993)',
 "Muriel's Wedding (1994)",
 'Nine Months (1995)',
 'Operation Dumbo Drop (1995)',
 'Piano, The (1993)',
 'Postman, The (Postino, Il) (1994)',
 'Ready to Wear (Pret-A-Porter) (1994)',
 'Remains of the Day, The (1993)',
 'Sabrina (1995)',
 "Schindler's List (1993)",
 'Secret Garden, The (1993)',
 'Sense and Sensibility (1995)',
 'Shadowlands (1993)',
 'Silence of the Lambs, The (1991)',
 'Star Trek: Generations (1994)',
 'Stargate (1994)']

In [47]:
len(user_random_watched_list)

33

Creating a DataFrame of the selected user's watch list

In [48]:
user_random_watched_df = users_movies_df[user_random_watched_list]
user_random_watched_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),Four Weddings and a Funeral (1994),Home Alone (1990),Jurassic Park (1993),Like Water for Chocolate (Como agua para chocolate) (1992),Little Women (1994),Mr. Holland's Opus (1995),Mrs. Doubtfire (1993),Much Ado About Nothing (1993),Muriel's Wedding (1994),Nine Months (1995),Operation Dumbo Drop (1995),"Piano, The (1993)","Postman, The (Postino, Il) (1994)",Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN
5.0,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,3.0,NaN,3.0,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Counting the number of items from this list that each of the users have watched

In [49]:
users_movies_count = user_random_watched_df.T.notnull().sum()
users_movies_count

userId
1.0          1
2.0          2
3.0          4
4.0          6
5.0         11
            ..
138489.0     1
138490.0     7
138491.0     0
138492.0     2
138493.0     9
Length: 138493, dtype: int64

Renaming the columns

In [50]:
users_movies_count = users_movies_count.reset_index()
users_movies_count.columns = ['userId', "watch_count"]
users_movies_count

,userId,watch_count
0,1.0,1
1,2.0,2
2,3.0,4
3,4.0,6
4,5.0,11
...,...,...
138488,138489.0,1
138489,138490.0,7
138490,138491.0,0
138491,138492.0,2


Reducing the DataFrame to focus on the users that have watched more than 60% of the movies that our selected user has watched

In [51]:
criterion = len(user_random_watched_list) * 0.6

users_movies_count[users_movies_count['watch_count'] > criterion].sort_values('watch_count', ascending=False)
users_movies_count

,userId,watch_count
0,1.0,1
1,2.0,2
2,3.0,4
3,4.0,6
4,5.0,11
...,...,...
138488,138489.0,1
138489,138490.0,7
138490,138491.0,0
138491,138492.0,2


Finding the User ID of those users

In [53]:
users_same_movies_IDs = users_movies_count[users_movies_count['watch_count'] > criterion]['userId']
users_same_movies_IDs

90            91.0
129          130.0
155          156.0
157          158.0
159          160.0
            ...   
138207    138208.0
138278    138279.0
138381    138382.0
138414    138415.0
138482    138483.0
Name: userId, Length: 4139, dtype: float64

Identifying users that have watched all the movies that our selected user has watched

In [54]:
users_movies_count[users_movies_count['watch_count'] == len(user_random_watched_list)]

,userId,watch_count
8404,8405.0,33
13937,13938.0,33
15918,15919.0,33
28940,28941.0,33
41388,41389.0,33
46662,46663.0,33
76629,76630.0,33
81217,81218.0,33
81595,81596.0,33
83089,83090.0,33


##### Now for recommendation, we need to identify users with a very similar watching experience to the selected user

Aggregating data of the selected user and other users

In [57]:
aggregate_df = pd.concat([user_random_watched_df[user_random_watched_df.index.isin(users_same_movies_IDs)], user_random_df[user_random_watched_list]])
aggregate_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),Four Weddings and a Funeral (1994),Home Alone (1990),Jurassic Park (1993),Like Water for Chocolate (Como agua para chocolate) (1992),Little Women (1994),Mr. Holland's Opus (1995),Mrs. Doubtfire (1993),Much Ado About Nothing (1993),Muriel's Wedding (1994),Nine Months (1995),Operation Dumbo Drop (1995),"Piano, The (1993)","Postman, The (Postino, Il) (1994)",Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
91.0,2.5,2.0,NaN,4.0,3.5,NaN,NaN,NaN,NaN,4.0,4.5,3.0,3.5,NaN,3.5,3.5,4.0,4.0,3.5,2.5,2.5,NaN,NaN,NaN,NaN,2.5,4.5,NaN,NaN,NaN,3.0,3.5,4.0
130.0,4.0,3.0,NaN,3.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,3.0,4.0,3.0,NaN,4.0,4.0,NaN,3.0,NaN,NaN,4.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN,3.0
156.0,3.0,NaN,NaN,5.0,5.0,3.0,NaN,NaN,4.0,5.0,3.0,4.0,5.0,NaN,NaN,4.0,5.0,4.0,3.0,3.0,4.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,4.0,4.0,5.0,3.0,4.0
158.0,2.0,1.0,4.0,4.0,3.0,5.0,NaN,4.0,NaN,5.0,4.0,3.0,3.0,NaN,NaN,3.0,3.0,NaN,5.0,3.0,3.0,NaN,5.0,NaN,5.0,3.0,5.0,5.0,4.0,5.0,5.0,NaN,NaN
160.0,NaN,NaN,3.0,5.0,4.0,4.0,NaN,3.0,2.0,5.0,5.0,4.0,4.0,NaN,4.0,NaN,4.0,3.0,NaN,3.0,NaN,3.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,5.0,4.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138279.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,4.0,NaN,5.0,4.0,2.0,5.0,5.0,5.0,4.0,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,3.0,3.0
138382.0,1.0,1.0,4.0,2.0,3.0,5.0,NaN,4.0,3.0,5.0,3.0,3.0,5.0,5.0,NaN,NaN,5.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,3.0,NaN,3.0,3.0,NaN,5.0,NaN,4.0
138415.0,1.0,NaN,3.0,4.0,3.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,3.0,3.0,3.0,4.0,4.0,NaN,4.0,4.0,NaN,3.0,5.0,NaN,5.0,NaN,5.0,3.0,4.0,4.0,NaN,3.0,NaN


Calculating the correlations for all users and creating a DataFrame for it

In [79]:
# CODE EXPLANATIONS: It first calculates the pairwise correlation between columns in the 
# aggregate_df DataFrame, sorts these correlations, and then removes duplicate entries 
# to create a Series (correlation_df) representing unique correlations in ascending order.
correlation_df = aggregate_df.T.corr().unstack().sort_values().drop_duplicates()

correlation_df = pd.DataFrame(correlation_df, columns=['correlation'])
correlation_df.index.names = ['user_1', 'user_2']
correlation_df = correlation_df.reset_index()

correlation_df

,user_1,user_2,correlation
0,78262.0,105664.0,-0.954490
1,55005.0,119715.0,-0.952579
2,137558.0,104652.0,-0.950000
3,72596.0,29398.0,-0.939060
4,28866.0,67756.0,-0.936065
...,...,...,...
5569300,32344.0,68063.0,0.977387
5569301,69856.0,15366.0,0.978167
5569302,138483.0,138483.0,1.000000
5569303,16562.0,41005.0,1.000000


Identifying users with a correlation higher than 0.65 with our randomly selected user

In [88]:
top_users_1 = correlation_df[(correlation_df['user_1'] == user_random_id) & (correlation_df['correlation'] >= 0.65)][['user_2', 'correlation']]
top_users_1.rename(columns={'user_2': 'userId'}, inplace=True)
top_users_1

,userId,correlation
5301384,105474.0,0.651830
5311750,94379.0,0.655596
5325482,132307.0,0.660726
5326814,42696.0,0.661268
5330538,118119.0,0.662693
5332159,103998.0,0.663340
5335899,129079.0,0.664777
5336773,61709.0,0.665115
5336774,61709.0,0.665115
5340002,10431.0,0.666356


In [89]:
len(top_users_1)

42

In [90]:
top_users_2 = correlation_df[(correlation_df['user_2'] == user_random_id) & (correlation_df['correlation'] >= 0.65)][['user_1', 'correlation']]
top_users_2.rename(columns={'user_1': 'userId'}, inplace=True)
top_users_2

,userId,correlation
5301383,105474.0,0.651830
5310742,82666.0,0.655212
5316505,50211.0,0.657333
5316506,50211.0,0.657333
5325483,132307.0,0.660726
5326813,42696.0,0.661268
5332160,103998.0,0.663340
5336669,73112.0,0.665066
5337606,1362.0,0.665461
5345236,26238.0,0.668515


In [91]:
len(top_users_2)

38

In [98]:
top_users = pd.concat([top_users_1, top_users_2]).drop_duplicates('userId').sort_values('correlation', ascending=False).reset_index(drop=True)
top_users

,userId,correlation
0,13477.0,0.802181
1,45158.0,0.800749
2,101628.0,0.790405
3,7542.0,0.772183
4,127259.0,0.763925
5,130316.0,0.758973
6,25507.0,0.752932
7,7438.0,0.752716
8,128241.0,0.752587
9,108007.0,0.751066


Finding the movies that these users (with high correlation to the selected user) have watched

In [99]:
top_users_ratings = top_users.merge(ratings[['userId', 'movieId', 'rating']], how='inner')
top_users_ratings

,userId,correlation,movieId,rating
0,13477.0,0.802181,1,4.0
1,13477.0,0.802181,2,3.0
2,13477.0,0.802181,3,2.0
3,13477.0,0.802181,5,3.0
4,13477.0,0.802181,7,5.0
...,...,...,...,...
23125,105474.0,0.651830,3791,3.0
23126,105474.0,0.651830,3798,5.0
23127,105474.0,0.651830,3809,4.0
23128,105474.0,0.651830,3840,1.0


### Creating a new variable 'weighted_rating' of both the correlation and rating values

In [102]:
top_users_ratings['weighted_rating'] = top_users_ratings['correlation'] * top_users_ratings['rating']

Creating the final DataFrame for recommendation with the mean weighted_rating for each movie title

In [105]:
recommendation_df = top_users_ratings.groupby('movieId').agg({'weighted_rating': 'mean'})
recommendation_df

,weighted_rating
movieId,
1,2.471265
2,1.655777
3,1.608621
4,1.691054
5,1.483689
...,...
112183,2.028250
112370,1.314665
112552,2.366291


In [106]:
recommendation_df = recommendation_df[recommendation_df['weighted_rating'] > 3.5].sort_values('weighted_rating', ascending=False).reset_index()
recommendation_df

,movieId,weighted_rating
0,53,3.952023
1,1922,3.763580
2,2057,3.763580
3,2485,3.763580
4,3118,3.763580
5,887,3.680055
6,2675,3.680055
7,1184,3.670461
8,6216,3.664714
9,7585,3.664714


In [107]:
recommendation_df.merge(movies[['movieId', 'title']])

,movieId,weighted_rating,title
0,53,3.952023,Lamerica (1994)
1,1922,3.763580,Whatever (1998)
2,2057,3.763580,"Incredible Journey, The (1963)"
3,2485,3.763580,She's All That (1999)
4,3118,3.763580,Tumbleweeds (1999)
5,887,3.680055,Talk of Angels (1998)
6,2675,3.680055,Twice Upon a Yesterday (a.k.a. Man with Rain i...
7,1184,3.670461,Mediterraneo (1991)
8,6216,3.664714,Nowhere in Africa (Nirgendwo in Afrika) (2001)
9,7585,3.664714,Summertime (1955)
